## Sandbox

Un  `.ipynb` pour tester des idées :)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));

In [ ]:
infos = open(
    readInstance,
    file,
);

In [ ]:
graph = graphFromData(infos);

In [ ]:
res = dualSolve(graph; timelimit=time() + 60.0)
println(mkPath(res.a))
println(res.obj_value)